# Package installation

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as plot
import plotly.graph_objects as go
import plotly.offline as py
import numpy as np 

# A description for each one of these sheets is given below:

* orders: it contains the orders that need the calculation of baskets and cold bags. Also, you can see when the order must be delivered and which warehouse received the order.

* order_products: contains what products and how many units of it were on an order from the sheet “orders”.

* store_products: contains information about the storage of the products and a marketing category.

* products: contains information about the products, its dimensions and weight, as well as their names and the can_mix category.


In [ ]:

orders = pd.read_excel('dataset.xlsx', sheet_name='orders')

In [ ]:
order_products = pd.read_excel('dataset.xlsx', sheet_name='order_products')

In [ ]:
store_products = pd.read_excel('dataset.xlsx', sheet_name='store_products')

In [ ]:
products = pd.read_excel('dataset.xlsx', sheet_name='products')

# Important Informations

### Calculations

$ {Total volume} = {height} * {width} * {length} $

$ {Cubage} = {Totalvolume} * {weight} $

### The baskets have the following characteristics:
*	Length: 50 centimeters
*	Width: 40 centimeters
*	Height: 60 centimeters

Total weight that it can resit: 25 kilograms


In [ ]:
total_volume_baskets = 50 * 40 * 60
cubage_baskets = total_volume_baskets  * (25 * 1000)
cubage_baskets

### The cold bags have the following characteristics:
* Length: 23 centimeters
* Width: 37 centimeters
* Height: 18 centimeters

Total weight that it can resit: 5 kilograms


In [ ]:
total_volume_coldbag = 23 * 37 * 18 
cubage_coldbag = total_volume_coldbag * (5 * 1000)
cubage_coldbag 

### A category that determines if the product can be mixed with other products
* Food
* Toilet
* Pets

### A storage type that determines the type of package that will be used for the product
* Dry
* Refrigerated
* Frozen


# Data Analysis.

## Data from Orders

In [ ]:
orders.sort_values(['order_id','warehouse_id'], ascending=False)
orders.head(3)

In [ ]:
print(orders.shape)
print()
print(orders.describe())
print()
print(orders.info())

In [ ]:
orders.isnull().sum()

In [ ]:
sns.heatmap(orders.isnull(),yticklabels=False)

## Data from Order Products

In [ ]:
order_products.sort_values(['order_id','store_product_id'], ascending=False)
order_products.head(3)

In [ ]:
print(order_products.shape)
print()
print(order_products.describe())
print()
print(order_products.info())

In [ ]:
order_products.isnull().sum()

In [ ]:
sns.heatmap(order_products.isnull(),yticklabels=False)

## Data from Store Products

In [ ]:
store_products.sort_values(['store_product_id','product_id','category'], ascending=False)
store_products.head(3)

In [ ]:
print(store_products.shape)
print()
print(store_products.describe())
print()
print(store_products.info())

In [ ]:
store_products.isnull().sum()

In [ ]:
sns.heatmap(store_products.isnull(),yticklabels=False)

## Analysing data from lines without values

In [ ]:
shape_before = store_products.shape[0]
shape_now = store_products.dropna().shape[0]
lines_rev = shape_before - shape_now
lines_per = (lines_rev/shape_before) * 100

print(f"Total of lines: {shape_before} Removed lines empty {lines_rev} percent of null: {round(lines_per, 2)}%")


## Data from Products

In [ ]:
products.sort_values(['product_id','product_name'], ascending=False)
products['prod_volcub'] = products['height'] * products['width'] * products['length'] 
products['prod_cubage'] = products['prod_volcub'] * products['weight'] 
products.drop('reference',axis=1, inplace=True)
products.astype({'height':float, 'width':float,	'length':float,	'weight':float}).dtypes
products.head(3)


In [ ]:
can_mix = products['can_mix'].unique()
can_mix 

In [ ]:
product_name = products['product_name'].drop_duplicates().sort_values()
[name for name in product_name ] 

In [ ]:
print(products.shape)
print()
print(products.describe())
print()
print(products.info())

In [ ]:
products.isnull().sum()

In [ ]:
sns.heatmap(products.isnull(),yticklabels=False)

## Analysing data from lines without values

In [ ]:
shape_before = products.shape[0]
shape_now = products.dropna().shape[0]
lines_rev = shape_before - shape_now
lines_per = (lines_rev/shape_before) * 100
perc_lines =  100-(shape_now*100/shape_before)

print(f"Total of lines: {shape_before} Removed lines empty {lines_rev} percent of null: {round(lines_per, 2)}%")

# Creating analysis for result output

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10000)

In [ ]:
df = orders.merge(order_products, how='left', on='order_id')

df.head(3)

In [ ]:
df = df.merge(store_products, how='left', on='store_product_id')
df.head(3)

In [ ]:
df = df.merge(products, how='left', on='product_id')
df.head(3)

In [ ]:
df2 = df[df['store_product_id'].isnull()]
df2

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df['product_id'] = df['product_id'].astype(int)

In [ ]:
df['baskets'] = 0
df['cold_bags'] = 0

In [ ]:
df.head(5)

In [ ]:
# Selecting order for analysi
df[df['order_id']==1781559]

* basket -> seco + categoria(food) diferente não pode ser misturado + cubage
* cold_bag -> Refrigerado and Congelado + categoria(food) diferente não pode ser misturado + cubage

In [ ]:
# Appling conditions for packing
df['baskets'] = np.where(((df['storage']=='Seco')&(df['can_mix']=='Food')|(df['can_mix']=='Toilet')|(df['can_mix']=='Pets')&(df['prod_cubage'] <= cubage_baskets)), 1, 0)
df['cold_bags'] = np.where(((df['storage']=='Refrigerado')|(df['storage']=='Congelado')&(df['can_mix']=='Food')|(df['can_mix']=='Toilet')|(df['can_mix']=='Pets')&(df['prod_cubage'] <= cubage_coldbag)), 1, 0)


In [ ]:
# Selecting data
df[df['order_id']==	1781560]  

In [ ]:
output_result = df[['order_id','baskets','cold_bags']]
output_result = output_result.groupby('order_id').sum()
output_result.loc['Soma'] = [output_result['baskets'].sum(axis=0),output_result['cold_bags'].sum(axis=0)] 
output_result

In [ ]:
output_result.to_excel("type_packing_orders.xlsx", sheet_name='type_packing')  

In [ ]:
bar_gra = output_result.loc[1781559:1808868]
bar_gra


In [ ]:
bar_gra.plot(kind='line', figsize=(20,10), title='Quantity of baskets and coldbag per order\n', fontsize=15)

In [ ]:
plt.figure(figsize=(50,10))
sns.countplot(bar_gra['baskets'])

In [ ]:
plt.figure(figsize=(50,10))
sns.countplot(bar_gra['cold_bags'])

In [ ]:
corr_df = bar_gra.corr(method='pearson')
plt.figure(figsize=(8, 6))
sns.heatmap(corr_df, annot=True)
plt.show()

In [ ]:
prod = df[['product_name', 'height', 'width', 'length', 'weight']]
prod

In [ ]:
corr_df = prod.corr(method='pearson')
plt.figure(figsize=(8, 6))
sns.heatmap(corr_df, annot=True)
plt.show()

In [ ]:
orders_id_dt = df[['order_id',	'delivery_date']]
orders_id_dt = orders_id_dt.drop_duplicates().reset_index(drop=True)
orders_id_dt 




In [ ]:
delivery_date = orders_id_dt['delivery_date'].value_counts()
delivery_date

In [ ]:
delivery_date.plot(kind='bar', figsize=(20,10), title='Quantity of order per date\n', fontsize=15)